In [2]:
from model import Model
from utils import fetch_dataset
from grakel.kernels import ShortestPath, VertexHistogram, EdgeHistogram

In [43]:
MUTAG = fetch_dataset("BZR", verbose=False)
SP = EdgeHistogram(normalize=False)
# SP.fit_transform(MUTAG.data)

In [41]:
for x in SP.X:
    print(x)

[32.  4.  2.  0.]
[22.  4.  2.  0.]
[22.  4.  2.  0.]
[34.  8.  2.  0.]
[12.  8.  2.  0.]
[38. 16.  8.  0.]
[24.  8.  2.  0.]
[24. 16.  4.  0.]
[ 0. 16. 10.  0.]
[24. 12.  2.  0.]
[32.  4.  2.  0.]
[38.  6.  2.  0.]
[34. 12.  4.  0.]
[ 0. 18. 10.  0.]
[38.  4.  2.  0.]
[34. 12.  4.  0.]
[12.  8.  2.  0.]
[24. 10.  4.  0.]
[22.  4.  2.  0.]
[24. 12.  2.  2.]
[24. 10.  4.  0.]
[24. 12.  2.  0.]
[42.  8.  4.  0.]
[60.  4.  2.  0.]
[24. 12.  2.  0.]
[12. 10.  4.  0.]
[48.  4.  2.  0.]
[24. 12.  2.  0.]
[24. 18.  8.  0.]
[42.  8.  4.  0.]
[34. 12.  4.  0.]
[24. 18.  8.  0.]
[24. 22.  4.  0.]
[22.  4.  2.  0.]
[24. 12.  2.  0.]
[22.  6.  2.  0.]
[24.  8.  4.  0.]
[24.  6.  2.  0.]
[22.  8.  2.  0.]
[22.  4.  2.  0.]
[24. 12.  2.  0.]
[ 0. 18. 10.  0.]
[34.  8.  2.  0.]
[24. 12.  2.  0.]
[12.  8.  4.  0.]
[38. 14.  2.  0.]
[24. 22.  4.  0.]
[32.  4.  2.  0.]
[32.  8.  4.  0.]
[22.  8.  4.  0.]
[38. 18.  4.  0.]
[38. 18.  4.  0.]
[24. 14.  4.  0.]
[24. 16.  2.  0.]
[22.  6.  2.  0.]
[32.  4.  

In [38]:
from collections import Counter
edges = [edge for edge in MUTAG.data[170][2].values()]
c = Counter(edges)
print(c)



Counter({0: 34, 1: 8, 2: 2})
